# 기상청 API 가공하기

In [95]:
import pandas as pd
import os
from typing import List
import numpy as np
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
import requests
import datetime
from datetime import datetime, timedelta
import math

In [96]:

def auto_get(start_t: str, end_t: str):
    start = start_t
    end = end_t
    d = '548'
    key = 'tHdgtN_vShS3YLTf74oUyg'

    url = f'https://apihub.kma.go.kr/api/typ01/cgi-bin/url/nph-aws2_min_vis?tm1={start}&tm2={end}&stn={d}&disp=0&help=1&authKey={key}'

    print(url, '\n')

    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html')
    front = soup.text.split('\n')

    real  = []
    tmp = []
    for i in range(len(front)-1): #마지막에 줄바꿈 문자가 있어서 인덱싱 에러가 나므로 마지막줄을 빼준다.
        if(front[i][0] != "#"):
            tmp = str(front[i]).split()
            #print(tmp)
            real.append(tmp)

    cul_names = ['time', 'stn', 'lon', 'lat', 's', 'v1', 'v10', 'weather', 'weather_15' ]
    real_df = pd.DataFrame(real, columns = cul_names)
    real_df.info()
    real_df.head()
    real_df.tail()

    real_df.to_csv(f'../outputs/{start[2:]}_{end[2:]}.csv', index = None) #csv파일로 생성

In [ ]:
months = ['01', '02']
days_1 = range(17, 31)
#days_2 = range(1, 29)
days_2 = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17',
          '18', '19', '20', '21','22', '23', '24', '25','26', '27', '28']


for i in months:
    if i == '01':
        for j in days_1:
            auto_get(f'2023{i}{j}0000', f'2023{i}{j}1159') #오전
            auto_get(f'2023{i}{j}1200', f'2023{i}{j}2359') #오후
    if i == '02':
        for j in days_2:
            auto_get(f'2023{i}{j}0000', f'2023{i}{j}1159') #오전
            auto_get(f'2023{i}{j}1200', f'2023{i}{j}2359') #오후

In [98]:
ddd = pd.read_csv(f'../outputs/2301170000_2301171159.csv', sep=",")
ddd.head()
ddd[['time', 'weather']].head()

,time,weather
0,202301170000,0
1,202301170001,0
2,202301170002,0
3,202301170003,0
4,202301170004,0


In [99]:
def weather_summery(value: int):
    if 0 <= value <=2: #dry 맑음
        return 0
    elif value == 4: #other 연무
        return 4
    elif value == 10: #other 박무
        return 4
    elif value == 30: #other 안개
        return 4
    elif 40 <= value <= 42: #rain 비
        return 1
    elif 50 <= value <= 59: #rain 안개비
        return 1
    elif 60 <= value <= 68: #rain 비
        return 1
    elif 71 <= value <= 76: #snow 눈
        return 2
    else:
        return 3 #err

In [100]:

for i in months:
    if i == '01':
        for j in days_1:
            data_am = pd.read_csv(f'../outputs/23{i}{j}0000_23{i}{j}1159.csv', sep=",")[['time', 'weather']]
            data_pm = pd.read_csv(f'../outputs/23{i}{j}1200_23{i}{j}2359.csv', sep=",")[['time', 'weather']]
            day = pd.concat([data_am, data_pm])
            day['weather'] = day['weather'].map(lambda x : weather_summery(x))
            day.to_csv(f'../outputs/day/2023{i}{j}.csv', index = None) #csv파일로 
    if i == '02':
        for j in days_2:
            data_am = pd.read_csv(f'../outputs/23{i}{j}0000_23{i}{j}1159.csv', sep=",")[['time', 'weather']]
            data_pm = pd.read_csv(f'../outputs/23{i}{j}1200_23{i}{j}2359.csv', sep=",")[['time', 'weather']]
            day = pd.concat([data_am, data_pm])
            day['weather'] = day['weather'].map(lambda x : weather_summery(x))
            day.to_csv(f'../outputs/day/2023{i}{j}.csv', index = None) #csv파일로 생성


In [101]:
files = os.listdir('../outputs/day')[1:]
csves = pd.read_csv('../outputs/day/20230117.csv', sep=",")

for i in range(len(files)):
    #print(f'../outputs/day/{files[i]}')
    csves = pd.concat([csves, pd.read_csv('../outputs/day/' + files[i], sep=",")])

csves.to_csv(f'../outputs/day/2023_2month.csv', index = None) #csv파일로

['20230117.csv', '20230118.csv', '20230119.csv', '20230120.csv', '20230121.csv', '20230122.csv', '20230123.csv', '20230124.csv', '20230125.csv', '20230126.csv', '20230127.csv', '20230128.csv', '20230129.csv', '20230130.csv', '20230201.csv', '20230202.csv', '20230203.csv', '20230204.csv', '20230205.csv', '20230206.csv', '20230207.csv', '20230208.csv', '20230209.csv', '20230210.csv', '20230211.csv', '20230212.csv', '20230213.csv', '20230214.csv', '20230215.csv', '20230216.csv', '20230217.csv', '20230218.csv', '20230219.csv', '20230220.csv', '20230221.csv', '20230222.csv', '20230223.csv', '20230224.csv', '20230225.csv', '20230226.csv', '20230227.csv', '20230228.csv', '2023_2month.csv']


# 이노로드 데이터 가공

In [102]:
#9.54	9.85
b = 17.62
c = 243.12
air_temperature = 9.54
r_humidity = 9.85

gamma = ( b * air_temperature / ( c + air_temperature ) ) + math.log( r_humidity / 100.0 )
dewpoint = ( c * gamma ) / ( b - gamma )
print(dewpoint)

-20.84489301570457


In [103]:

def coculate_dewpoint(row) -> float:
    ##### 이슬점 근사치를 계산한다.
    ## 입력 : 온도, 습도
    ## 출력 : 이슬점 온도
    
    ## 상수 ## 
    b = 17.62
    c = 243.12
    air_temperature = row['air_temperature']
    r_humidity = row['r_humidity']
    gamma = ( b * air_temperature / ( c + air_temperature ) ) + math.log( r_humidity / 100.0 )
    dewpoint = ( c * gamma ) / ( b - gamma )
    
    return dewpoint

In [104]:
def time_tf(i_time: str):
    
    if len(i_time) != 12:
        if int(i_time.split()[1].split(':')[0]) < 10:
            i_time = f'{i_time[:10]} 0{i_time[-4:]}'

        date_ = datetime.strptime(i_time, "%Y-%m-%d %H:%M")
        date_ = date_.strftime("%Y%m%d%H%M")
        return date_
    
    return i_time

In [105]:
sf_1 = pd.read_csv('../processed_data/sf_t_07_1m.csv', sep=",") #노면온도1월
sf_2 = pd.read_csv('../processed_data/sf_t_07_2m.csv', sep=",") #노면온도2월
csves = pd.concat([sf_1, sf_2])
csves.to_csv(f'../processed_data/sf_t_07_1m&2m.csv', index = None) #csv파일로

In [106]:
#이노로드 데이터 종류별로 불러오기
er_all = pd.read_csv('../processed_data/es_all_1m&2m.csv', sep=",") #기상데이터
er_rh = pd.read_csv('../processed_data/es_h_1m&2m.csv', sep=",") #습도(여유분 데이터)
er_sf = pd.read_csv('../processed_data/sf_t_07_1m&2m.csv', sep=",") #노면온도

In [107]:
print(er_all.head())
er_all['time'] = er_all['time'].map(lambda x : time_tf(x))
print(er_all.head())

              time  wind_deg  wind_velocity  air_temperature  r_humidity  \
0  2023-03-09 8:08      69.0            0.3             10.3        75.9   
1  2023-03-09 8:06      86.0            0.4             10.1        76.1   
2  2023-03-09 8:04       3.0            0.3             10.1        76.7   
3  2023-03-09 8:02       1.0            0.2             10.1        77.0   
4  2023-03-09 8:00       1.0            0.3             10.1        75.7   

   rain_  
0    1.9  
1    1.9  
2    1.9  
3    1.9  
4    1.9  
           time  wind_deg  wind_velocity  air_temperature  r_humidity  rain_
0  202303090808      69.0            0.3             10.3        75.9    1.9
1  202303090806      86.0            0.4             10.1        76.1    1.9
2  202303090804       3.0            0.3             10.1        76.7    1.9
3  202303090802       1.0            0.2             10.1        77.0    1.9
4  202303090800       1.0            0.3             10.1        75.7    1.9


In [108]:
print(er_rh.head())
er_rh['time'] = er_rh['time'].map(lambda x : time_tf(x))
print(er_rh.head())

              time  surface_rh
0  2023-03-09 9:10          73
1  2023-03-09 9:08          73
2  2023-03-09 9:06          74
3  2023-03-09 9:04          74
4  2023-03-09 9:02          75
           time  surface_rh
0  202303090910          73
1  202303090908          73
2  202303090906          74
3  202303090904          74
4  202303090902          75


In [109]:
print(er_sf.head())
er_sf['time'] = er_sf['time'].map(lambda x : time_tf(x))
print(er_sf.head())

              time  surface_tmp
0  2023-01-01 0:00         -5.2
1  2023-01-01 0:02         -5.1
2  2023-01-01 0:04         -4.9
3  2023-01-01 0:06         -4.8
4  2023-01-01 0:08         -4.9
           time  surface_tmp
0  202301010000         -5.2
1  202301010002         -5.1
2  202301010004         -4.9
3  202301010006         -4.8
4  202301010008         -4.9


# 데이터 병합가공

In [110]:
#이노로드 데이터 종류별로 불러오기
env = pd.read_csv('../processed_data/es_all_1m&2m.csv', sep=",") #기상데이터
rrh = pd.read_csv('../processed_data/es_h_1m&2m.csv', sep=",") #습도(여유분 데이터)
sft = pd.read_csv('../processed_data/sf_t_07_1m&2m.csv', sep=",") #노면온도

#기상청 데이터 1, 2월분
wea = pd.read_csv('../processed_data/2023_2month.csv', sep=",") #날씨코드만
wea['time'] = wea['time'].astype(str)

In [111]:
env_sft = pd.merge(er_all, er_sf, left_on='time', right_on='time')
env_sft.head()

,time,wind_deg,wind_velocity,air_temperature,r_humidity,rain_,surface_tmp
0,202302282358,12.0,0.2,3.0,89.7,1.9,7.9
1,202302282356,12.0,0.4,3.0,89.8,1.9,7.9
2,202302282354,39.0,0.5,3.0,90.0,1.9,7.9
3,202302282352,91.0,1.1,3.0,90.4,1.9,7.9
4,202302282350,84.0,0.3,2.9,90.6,1.9,8.1


In [112]:
env_sft_wea = pd.merge(env_sft, wea, left_on='time', right_on='time')
env_sft_wea.head()

,time,wind_deg,wind_velocity,air_temperature,r_humidity,rain_,surface_tmp,weather
0,202302282358,12.0,0.2,3.0,89.7,1.9,7.9,0
1,202302282356,12.0,0.4,3.0,89.8,1.9,7.9,1
2,202302282354,39.0,0.5,3.0,90.0,1.9,7.9,1
3,202302282352,91.0,1.1,3.0,90.4,1.9,7.9,1
4,202302282350,84.0,0.3,2.9,90.6,1.9,8.1,0


In [113]:
env_sft_wea = env_sft_wea[::-1].reset_index(drop=True)
env_sft_wea.head()

,time,wind_deg,wind_velocity,air_temperature,r_humidity,rain_,surface_tmp,weather
0,202301171016,0.8,0.04,9.54,9.85,0.0,0.5,0
1,202301171018,35.6,0.02,9.54,9.69,0.0,0.6,0
2,202301171020,34.7,0.02,9.54,9.70,0.0,0.6,0
3,202301171150,0.0,0.00,9.67,6.74,0.0,4.7,0
4,202301171151,0.1,0.03,9.68,7.01,0.0,4.8,0


In [115]:
env_sft_wea['dewpoint'] = env_sft_wea.apply(coculate_dewpoint, axis=1)
env_sft_wea['uv'] = 0.0
if 'rain_' in env_sft_wea.columns:
    env_sft_wea.drop(labels='rain_',axis=1, inplace=True)
env_sft_wea.head()
env_sft_wea.to_csv(f'../processed_data/1m&2m_env_sft_wea.csv', index = None) #csv파일로

# 시간 기준으로 조회

In [116]:
now = datetime.now()
f_1h = (now + timedelta(minutes=60)).strftime('%Y%m%d%H%M')
f_2h = (now + timedelta(minutes=120)).strftime('%Y%m%d%H%M')

print(now.strftime('%Y%m%d%H%M'))
print(f_1h, f_2h)

202309150209
202309150309 202309150409


# 조회된 컬럼 이어 붙이기

In [117]:
data = pd.read_csv('../processed_data/1m&2m_env_sft_wea.csv', sep=",")
data['time'] = data['time'].astype(str)
data.head()

,time,wind_deg,wind_velocity,air_temperature,r_humidity,surface_tmp,weather,dewpoint,uv
0,202301171016,0.8,0.04,9.54,9.85,0.5,0,-20.844893,0.0
1,202301171018,35.6,0.02,9.54,9.69,0.6,0,-21.033614,0.0
2,202301171020,34.7,0.02,9.54,9.70,0.6,0,-21.021738,0.0
3,202301171150,0.0,0.00,9.67,6.74,4.7,0,-25.039533,0.0
4,202301171151,0.1,0.03,9.68,7.01,4.8,0,-24.595119,0.0


In [118]:
if 'rain_' in data.columns:
    data.drop(labels='rain_',axis=1, inplace=True)
data.head()

,time,wind_deg,wind_velocity,air_temperature,r_humidity,surface_tmp,weather,dewpoint,uv
0,202301171016,0.8,0.04,9.54,9.85,0.5,0,-20.844893,0.0
1,202301171018,35.6,0.02,9.54,9.69,0.6,0,-21.033614,0.0
2,202301171020,34.7,0.02,9.54,9.70,0.6,0,-21.021738,0.0
3,202301171150,0.0,0.00,9.67,6.74,4.7,0,-25.039533,0.0
4,202301171151,0.1,0.03,9.68,7.01,4.8,0,-24.595119,0.0


In [119]:
left = data.iloc[2]['time']
left

'202301171020'

In [ ]:
cols = { 'time' : 'time_1h',
    'wind_deg' : 'wind_deg_1h', 
    'wind_velocity' : 'wind_velocity_1h', 	
    'air_temperature' : 'air_temperature_1h', 
    'r_humidity' : 'r_humidity_1h',
    #'rain_' : 'rain_1h',
    'surface_tmp' : 'surface_tmp_1h',
    'weather' : 'weather_1h',
    'dewpoint' : 'dewpoint_1h',
    'uv' : 'uv_1h' 
    }
all_ = pd.DataFrame(columns=['time',
                            'wind_deg',
                            'wind_velocity',
                            'air_temperature',
                            'r_humidity',
                            #'rain_',
                            'surface_tmp',
                            'weather',
                            'dewpoint',
                            'uv',
                            'time_1h',
                            'wind_deg_1h',
                            'wind_velocity_1h',
                            'air_temperature_1h',
                            'r_humidity_1h',
                            #'rain_1h',
                            'surface_tmp_1h',
                            'weather_1h',
                            'dewpoint_1h',
                            'uv_1h'
                            ])

for i in range(data.shape[0]):
    left = data.iloc[i]
    t = datetime.strptime(left['time'], "%Y%m%d%H%M")
    f_1h = (t + timedelta(minutes=60)).strftime('%Y%m%d%H%M')
    target = data[data.time == f_1h]

    if target.empty:
        print(i, left['time'], f_1h, target.empty)
        pass
    else:
        #print(i, left['time'], f_1h)
        df1 = left.to_frame().T
        df1.index = [0]

        target.rename(columns=cols, inplace=True)
        df2 = target.iloc[0].to_frame().T
        df2.index = [0]

        result_df = pd.concat([df1, df2], axis=1)
        all_ = pd.concat([all_, result_df], axis=0)
        pass

all_.to_csv(f'../processed_data/input_and_1h.csv', index = None) #csv파일로

In [121]:
all_

,time,wind_deg,wind_velocity,air_temperature,r_humidity,surface_tmp,weather,dewpoint,uv,time_1h,wind_deg_1h,wind_velocity_1h,air_temperature_1h,r_humidity_1h,surface_tmp_1h,weather_1h,dewpoint_1h,uv_1h
0,202301171150,0,0,9.67,6.74,4.7,0,-25.0395,0,202301171250,0.9,0.04,9.87,4.96,9.1,0,-28.2474,0
0,202301171154,0.7,0.03,9.7,6.7,5.2,0,-25.0833,0,202301171254,6.6,0.06,9.92,5.25,9.5,0,-27.597,0
0,202301171156,0,0,9.72,6.12,5.7,0,-26.0688,0,202301171256,5.5,0.12,9.9,5.34,9.5,0,-27.427,0
0,202301171158,1.5,0.03,9.77,6.09,6.2,0,-26.086,0,202301171258,3.3,0.08,9.9,5.11,9.7,0,-27.9041,0
0,202301171200,35.9,0.06,9.77,5.52,6.2,0,-27.1612,0,202301171300,4.4,0.06,9.92,5.31,9.7,0,-27.4737,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,202302282250,0,0,1.9,91.6,7.8,0,0.676526,0,202302282350,84,0.3,2.9,90.6,8.1,0,1.5131,0
0,202302282252,0,0,1.9,91.9,8,0,0.721902,0,202302282352,91,1.1,3,90.4,7.9,1,1.58108,0
0,202302282254,0,0,2,92.3,8,0,0.881289,0,202302282354,39,0.5,3,90,7.9,1,1.51911,0
0,202302282256,175,0.3,2.2,93.3,8,0,1.22957,0,202302282356,12,0.4,3,89.8,7.9,1,1.48803,0


In [ ]:
for i in all_.columns:
    print(i)
    print(all_[all_[i].isnull()])
    print()

In [ ]:
cols2 = { 'time' : 'time_2h',
    'wind_deg' : 'wind_deg_2h', 
    'wind_velocity' : 'wind_velocity_2h', 	
    'air_temperature' : 'air_temperature_2h', 
    'r_humidity' : 'r_humidity_2h',
    #'rain_' : 'rain_2h',
    'surface_tmp' : 'surface_tmp_2h',
    'weather' : 'weather_2h',
    'dewpoint' : 'dewpoint_2h',
    'uv' : 'uv_2h' 
    }
all_ = pd.DataFrame(columns=['time',
                            'wind_deg',
                            'wind_velocity',
                            'air_temperature',
                            'r_humidity',
                            #'rain_',
                            'surface_tmp',
                            'weather',
                            'dewpoint',
                            'uv',
                            'time_2h',
                            'wind_deg_2h',
                            'wind_velocity_2h',
                            'air_temperature_2h',
                            'r_humidity_2h',
                            #'rain_2h',
                            'surface_tmp_2h',
                            'weather_2h',
                            'dewpoint_2h',
                            'uv_2h'
                            ])

for i in range(data.shape[0]):
    left = data.iloc[i]
    t = datetime.strptime(left['time'], "%Y%m%d%H%M")
    f_2h = (t + timedelta(minutes=120)).strftime('%Y%m%d%H%M')
    target = data[data.time == f_2h]

    if target.empty:
        print(i, left['time'], f_2h, target.empty)
        pass
    else:
        #print(i, left['time'], f_2h)
        df1 = left.to_frame().T
        df1.index = [0]

        target.rename(columns=cols2, inplace=True)
        df2 = target.iloc[0].to_frame().T
        df2.index = [0]

        result_df = pd.concat([df1, df2], axis=1)
        all_ = pd.concat([all_, result_df], axis=0)
        pass

all_.to_csv(f'../processed_data/input_and_2h.csv', index = None) #csv파일로

In [123]:
all_
#result_df

,time,wind_deg,wind_velocity,air_temperature,r_humidity,surface_tmp,weather,dewpoint,uv,time_2h,wind_deg_2h,wind_velocity_2h,air_temperature_2h,r_humidity_2h,surface_tmp_2h,weather_2h,dewpoint_2h,uv_2h
0,202301171016,0.8,0.04,9.54,9.85,0.5,0,-20.8449,0,202301171216,0.5,0.06,9.78,5.89,6.2,0,-26.4452,0
0,202301171018,35.6,0.02,9.54,9.69,0.6,0,-21.0336,0,202301171218,4.1,0.08,9.78,6.06,6,0,-26.1329,0
0,202301171020,34.7,0.02,9.54,9.7,0.6,0,-21.0217,0,202301171220,7.9,0.07,9.78,5.92,6,0,-26.3895,0
0,202301171150,0,0,9.67,6.74,4.7,0,-25.0395,0,202301171350,1.5,0.06,9.95,5.55,11.7,0,-26.9708,0
0,202301171154,0.7,0.03,9.7,6.7,5.2,0,-25.0833,0,202301171354,8.1,0.06,9.95,4.85,12.1,0,-28.4313,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,202302282150,0,0,2.7,90,8.7,0,1.2227,0,202302282350,84,0.3,2.9,90.6,8.1,0,1.5131,0
0,202302282152,0,0,2.7,89.9,8.7,0,1.20721,0,202302282352,91,1.1,3,90.4,7.9,1,1.58108,0
0,202302282154,0,0,2.7,89.8,8.7,0,1.1917,0,202302282354,39,0.5,3,90,7.9,1,1.51911,0
0,202302282156,281,0.3,2.7,90.1,8.5,0,1.23818,0,202302282356,12,0.4,3,89.8,7.9,1,1.48803,0


In [127]:
for i in all_.columns:
    print(i)
    print(all_[all_[i].isnull()])
    print()

time
Empty DataFrame
Columns: [time, wind_deg, wind_velocity, air_temperature, r_humidity, surface_tmp, weather, dewpoint, uv, time_2h, wind_deg_2h, wind_velocity_2h, air_temperature_2h, r_humidity_2h, surface_tmp_2h, weather_2h, dewpoint_2h, uv_2h]
Index: []

wind_deg
Empty DataFrame
Columns: [time, wind_deg, wind_velocity, air_temperature, r_humidity, surface_tmp, weather, dewpoint, uv, time_2h, wind_deg_2h, wind_velocity_2h, air_temperature_2h, r_humidity_2h, surface_tmp_2h, weather_2h, dewpoint_2h, uv_2h]
Index: []

wind_velocity
Empty DataFrame
Columns: [time, wind_deg, wind_velocity, air_temperature, r_humidity, surface_tmp, weather, dewpoint, uv, time_2h, wind_deg_2h, wind_velocity_2h, air_temperature_2h, r_humidity_2h, surface_tmp_2h, weather_2h, dewpoint_2h, uv_2h]
Index: []

air_temperature
Empty DataFrame
Columns: [time, wind_deg, wind_velocity, air_temperature, r_humidity, surface_tmp, weather, dewpoint, uv, time_2h, wind_deg_2h, wind_velocity_2h, air_temperature_2h, r_humi